In [7]:
from langgraph.graph import START, END, StateGraph
from typing import TypedDict, Literal, Annotated
from langchain_groq import ChatGroq
from langchain.messages import HumanMessage, SystemMessage

In [9]:
class TweetState(TypedDict):

    topic: str
    tweet: str
    evaluation: Literal["approved", "needs_improvement"]
    feedback: str
    iteration: int 
    max_iteration: int

In [17]:
from pydantic import BaseModel, Field

class TweetEvaluation(BaseModel):
    evaluation: Literal["approved", "needs_improvement"] = Field(..., description="Final evaluation result.")
    feedback: str = Field(..., description="feedback for the tweet.")

In [66]:
model_name = "qwen/qwen3-32b"
model = ChatGroq(model=model_name)

structured_evaluator_llm = model.with_structured_output(TweetEvaluation})


SyntaxError: closing parenthesis '}' does not match opening parenthesis '(' (122199953.py, line 4)

In [67]:
def generate_tweet(state: TweetState):

    messages = [
        SystemMessage(content="You're a funny and clever Tweeter/ X influencer"),
        HumanMessage(content=f"""
Write a short, original and hilarious tweet on the topic: "{state['topic']}."
Rules:
- Do not use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in meme logic, punchlines, or relatable takes.
- Use simple day to day English.
""")
    ]

    response = model.invoke(messages).content

    return {"tweet": response}

In [73]:
def evaluate_tweet(state: TweetState):

    messages = [
        SystemMessage(
            content="Return ONLY JSON with fields: evaluation, feedback."
        ),
        HumanMessage(
            content=f"""
Evaluate this tweet:

"{state['tweet']}"

Evaluation rules:
- originality
- humor
- punchiness
- virality
- format
- auto-reject if Q&A or >280 chars

Respond ONLY as JSON.
"""
        )
    ]

    response = structured_evaluator_llm.invoke(
        messages,
        response_format={"type": "json_object"}   # ← REQUIRED FOR GROQ
    )

    return {
        "evaluation": response.evaluation,
        "feedback": response.feedback
    }


In [74]:
def optimize_tweet(state: TweetState):

    messages = [
        SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
        HumanMessage(content=f"""
Improve the tweet based on this feedback:
"{state['feedback']}"

Topic: "{state['topic']}"
Original Tweet:
{state['tweet']}

Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 280 characters.
""")
    ]

    response = model.invoke(messages).content
    iteration = state['iteration'] + 1

    return {'tweet': response, 'iteration': iteration, 'tweet_history': [response]}

In [75]:
def route_evaluation(state: TweetState):
    if state['evaluation'] == "approved" or state['max_iteration'] > state['max_iteration']:
        return "approved"
    else:
        return "needs_improvement"

In [76]:
graph = StateGraph(TweetState)

graph.add_node("generate", generate_tweet)
graph.add_node("evaluate", evaluate_tweet)
graph.add_node("optimize", optimize_tweet)

graph.add_edge(START, "generate")
graph.add_edge("generate", "evaluate")

graph.add_conditional_edges("evaluate", route_evaluation, {"approved": END, "needs_improvement": "optimize"})
graph.add_edge("optimize", "evaluate")

workflow = graph.compile()

initial_state = {
    "topic": "Pakistan Railways",
    "iteration": 1,
    "max_iterations": 5
}

In [77]:
workflow.invoke(initial_state)

BadRequestError: Error code: 400 - {'error': {'message': 'json mode cannot be combined with tool/function calling', 'type': 'invalid_request_error', 'param': 'response_format'}}